<a href="https://colab.research.google.com/github/ruby0322/bda-midterm-project/blob/preprocessing/bda_midterm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境設定及整合


### Python 套件導入

In [36]:
# !pip install monpa
!pip install TCSP
!pip install jieba

In [37]:
import os
import pandas as pd
import chardet

# Vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import chi2

import numpy as np
from datetime import datetime, timedelta

In [38]:
# 中文處理
# import monpa
# from monpa import utils
import re
from TCSP import read_stopwords_list
import jieba

### 資料集下載及讀取

In [32]:
!gdown 1jJGJjXcQqRaAWBoHB4hf0PT-OH4vRvvM

Downloading...
From (original): https://drive.google.com/uc?id=1jJGJjXcQqRaAWBoHB4hf0PT-OH4vRvvM
From (redirected): https://drive.google.com/uc?id=1jJGJjXcQqRaAWBoHB4hf0PT-OH4vRvvM&confirm=t&uuid=215cdcb4-54fd-46a9-972c-9afef260178d
To: /content/BDA.zip
100% 470M/470M [00:05<00:00, 80.5MB/s]


In [33]:
!unzip -P bda2024 ./BDA.zip
!mkdir -p data
!mv ./*.csv ./data

Archive:  ./BDA.zip
  inflating: bda2024_202203-202402_內容數據_新聞1.csv  
  inflating: bda2024_202203-202402_內容數據_新聞2.csv  
  inflating: bda2024_202203-202402_內容數據_新聞3.csv  
  inflating: bda2024_202203-202402_討論數據_dcard.csv  
  inflating: bda2024_202203-202402_討論數據_mobile01-1.csv  
  inflating: bda2024_202203-202402_討論數據_mobile01-2.csv  
  inflating: bda2024_202203-202402_討論數據_ptt.csv  
  inflating: bda2024_微股力_個股交易數據-2年.csv  
  inflating: bda2024_微股力_社群PKTD-2年.csv  
  inflating: bda2024_微股力_籌碼數據-2年.csv  
  inflating: bda2024_微股力_財報數據-2年.csv  


In [34]:
root = '/content/data'
data = {}
for filename in os.listdir(root):
    words = filename.split('_')
    readable_filename = ''.join(words[-2:])
    filepath = f'{root}/{filename}'
    print(f'正在載入 {readable_filename} ...')
    df = pd.read_csv(filepath, encoding='utf-8')
    print(df)
    data[readable_filename[:-4]] = df
    print(f'成功載入 {readable_filename}。')

正在載入 微股力個股交易數據-2年.csv ...


<ipython-input-34-18a6a9e52baa>:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')


        stock_name stock_symbol   open   high    low  close  volume  \
0               日馳         1526  47.55  48.45  47.55  48.30     138   
1               日馳         1526  48.30  48.30  47.40  47.95     153   
2               日馳         1526  48.45  48.70  47.80  48.10     120   
3               日馳         1526  47.95  47.95  47.55  47.60     165   
4               日馳         1526  47.65  47.65  45.30  45.65     514   
...            ...          ...    ...    ...    ...    ...     ...   
1154220         統一         1216  76.20  78.60  76.00  77.20   20607   
1154221         統一         1216  76.80  78.80  76.40  78.80   16466   
1154222         統一         1216  78.00  78.70  77.10  78.20   16015   
1154223         統一         1216  78.00  78.00  76.30  77.40    8524   
1154224         統一         1216  76.10  77.20  75.80  76.20    8347   

                        date  
0        2022-03-01 00:00:00  
1        2022-03-02 00:00:00  
2        2022-03-03 00:00:00  
3        2022-03-04 00:

<ipython-input-34-18a6a9e52baa>:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')


         stock_name stock_symbol  foreign_investor_bought  \
0                信大         1109                     2000   
1                信大         1109                        0   
2                信大         1109                    11000   
3                信大         1109                    13000   
4                信大         1109                    68000   
...             ...          ...                      ...   
998026          現觀科         6906                    28000   
998027        金萬林-創         6645                    14000   
998028  群益ESG投等債20+       00937B                  1511400   
998029         玖鼎電力         4588                   389000   
998030      世界健身-KY         2762                    10000   

        foreign_investor_sold  investment_trust_bought  investment_trust_sold  \
0                        3000                        0                      0   
1                       11000                        0                      0   
2                       

In [35]:
data.keys()

dict_keys(['微股力個股交易數據-2年', '內容數據新聞2', '討論數據mobile01-2', '微股力籌碼數據-2年', '微股力財報數據-2年', '討論數據dcard', '微股力社群PKTD-2年', '討論數據mobile01-1', '內容數據新聞3', '討論數據ptt', '內容數據新聞1'])

### 列出各 table 的各行標題名稱

In [39]:
print('All tables:', data.keys(),'\n')
keys_list = list(data.keys())
for i in range(len(data)):
    print('Categories of', keys_list[i], ':', data[keys_list[i]].keys(),'\n')

All tables: dict_keys(['微股力個股交易數據-2年', '內容數據新聞2', '討論數據mobile01-2', '微股力籌碼數據-2年', '微股力財報數據-2年', '討論數據dcard', '微股力社群PKTD-2年', '討論數據mobile01-1', '內容數據新聞3', '討論數據ptt', '內容數據新聞1']) 

Categories of 微股力個股交易數據-2年 : Index(['stock_name', 'stock_symbol', 'open', 'high', 'low', 'close', 'volume',
       'date'],
      dtype='object') 

Categories of 內容數據新聞2 : Index(['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author',
       'content', 'page_url'],
      dtype='object') 

Categories of 討論數據mobile01-2 : Index(['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author',
       'content', 'page_url', 'content_type'],
      dtype='object') 

Categories of 微股力籌碼數據-2年 : Index(['stock_name', 'stock_symbol', 'foreign_investor_bought',
       'foreign_investor_sold', 'investment_trust_bought',
       'investment_trust_sold', 'dealer_bought', 'dealer_sold', 'date'],
      dtype='object') 

Categories of 微股力財報數據-2年 : Index(['stock_name', 'stock_symbol', 'period', 'gross_prof

In [40]:
# 類股代號皆轉為 string 型態
data['微股力個股交易數據-2年']['stock_symbol'] = data['微股力個股交易數據-2年']['stock_symbol'].astype(str)
data['微股力社群PKTD-2年']['stock_symbol'] = data['微股力社群PKTD-2年']['stock_symbol'].astype(str)
data['微股力籌碼數據-2年']['stock_symbol'] = data['微股力籌碼數據-2年']['stock_symbol'].astype(str)
data['微股力財報數據-2年']['stock_symbol'] = data['微股力財報數據-2年']['stock_symbol'].astype(str)

In [41]:
# 呼叫次數整理
data['微股力社群PKTD-2年']['actionP'] = data['微股力社群PKTD-2年']['actionP'] // 123
data['微股力社群PKTD-2年']['actionK'] = data['微股力社群PKTD-2年']['actionK'] // 123
data['微股力社群PKTD-2年']['actionT'] = data['微股力社群PKTD-2年']['actionT'] // 123
data['微股力社群PKTD-2年']['actionD'] = data['微股力社群PKTD-2年']['actionD'] // 123

### 依照查詢到的 AI 類股代號，將與股票漲跌有關的 table 進行篩選 （以 yahoo 股市為準）

In [42]:
# AI 類股代號
AI_stock_label = ['1504','2059','2308','2312','2317','2330','2353','2356','2357','2368',
          '2376','2377','2382','2383','2395','2439','2449','2453','2454','3029',
          '3037','3044','3078','3169','3227','3231','3324','3443','3653','3661',
          '4966','5274','5289','5351','5439','6166','6214','6274','6414','6515',
          '6643','6669','6689','6811','6841','6902','6933','8210']

In [43]:
print('AI 類別股_個股交易數據')
ai_stock_exchange_data = data['微股力個股交易數據-2年'][data['微股力個股交易數據-2年']['stock_symbol'].isin(AI_stock_label)]
#ai_stock_exchange_data = ai_stock_exchange_data.reset_index()
display(ai_stock_exchange_data)
print('\n')

print('AI 類別股_社群PKTD')
ai_stock_PKTD = data['微股力社群PKTD-2年'][data['微股力社群PKTD-2年']['stock_symbol'].isin(AI_stock_label)]
#ai_stock_PKTD = ai_stock_PKTD.reset_index()
display(ai_stock_PKTD)
print('\n')

print('AI 類別股_籌碼數據')
ai_stock_chips = data['微股力籌碼數據-2年'][data['微股力籌碼數據-2年']['stock_symbol'].isin(AI_stock_label)]
#ai_stock_chips = ai_stock_chips.reset_index()
display(ai_stock_PKTD)
print('\n')

print('AI 類別股_財報數據')
ai_stock_fin_state = data['微股力財報數據-2年'][data['微股力財報數據-2年']['stock_symbol'].isin(AI_stock_label)]
#ai_stock_fin_state = ai_stock_fin_state.reset_index()
display(ai_stock_fin_state)
print('\n')



AI 類別股_個股交易數據


,stock_name,stock_symbol,open,high,low,close,volume,date
3046,東元,1504,30.35,30.50,30.10,30.45,1108,2022-03-01 00:00:00
3978,東元,1504,30.20,30.40,30.05,30.05,684,2022-03-02 00:00:00
3979,東元,1504,30.20,30.35,30.10,30.20,612,2022-03-03 00:00:00
3980,東元,1504,30.00,30.25,29.65,29.80,4203,2022-03-04 00:00:00
3981,東元,1504,29.60,29.60,29.05,29.10,3301,2022-03-07 00:00:00
...,...,...,...,...,...,...,...,...
1127996,穎崴,6515,806.00,812.00,781.00,785.00,1604,2023-06-16 00:00:00
1127997,穎崴,6515,785.00,808.00,770.00,796.00,1192,2023-06-19 00:00:00
1127998,穎崴,6515,799.00,805.00,772.00,775.00,1065,2023-06-20 00:00:00
1127999,穎崴,6515,780.00,784.00,767.00,770.00,735,2023-06-21 00:00:00




AI 類別股_社群PKTD


,date,stock_symbol,actionP,actionK,actionT,actionD
36,2022-01-01 00:53,2449,0,1,0,0
41,2022-01-01 00:54,2449,0,0,1,0
63,2022-01-01 01:22,2357,1,0,0,0
82,2022-01-01 01:39,2308,0,0,1,0
102,2022-01-01 02:01,2368,0,0,1,0
...,...,...,...,...,...,...
10436687,2023-12-31 14:05,2330,0,1,0,0
10436688,2023-12-31 14:05,2330,1,0,0,0
10436701,2023-12-31 14:26,2454,0,1,0,0
10436702,2023-12-31 14:27,2454,1,0,0,0




AI 類別股_籌碼數據


,date,stock_symbol,actionP,actionK,actionT,actionD
36,2022-01-01 00:53,2449,0,1,0,0
41,2022-01-01 00:54,2449,0,0,1,0
63,2022-01-01 01:22,2357,1,0,0,0
82,2022-01-01 01:39,2308,0,0,1,0
102,2022-01-01 02:01,2368,0,0,1,0
...,...,...,...,...,...,...
10436687,2023-12-31 14:05,2330,0,1,0,0
10436688,2023-12-31 14:05,2330,1,0,0,0
10436701,2023-12-31 14:26,2454,0,1,0,0
10436702,2023-12-31 14:27,2454,1,0,0,0




AI 類別股_財報數據


,stock_name,stock_symbol,period,gross_profit_margin,operating_profit_margin,net_profit_margin,return_on_equity,debt_ratio,interest_coverage,current_ratio,quick_ratio,accounts_turnover,inventory_turnover,eps,book_value_per_share,date
461,東元,1504,Q,24.35,11.39,11.70,1.96,32.51,37.09,201.10,135.21,1.30,0.88,0.78,40.37,2023-03-01 00:00:00
462,東元,1504,Q,22.98,8.78,7.47,1.34,31.82,34.54,222.11,149.00,1.30,0.88,0.48,37.94,2022-12-01 00:00:00
464,東元,1504,Q,22.54,8.87,8.38,1.53,33.72,28.25,224.43,146.93,1.26,0.87,0.51,35.61,2022-09-01 00:00:00
466,東元,1504,Q,22.97,8.85,8.05,1.44,34.95,32.33,199.52,131.74,1.28,0.86,0.48,35.45,2022-06-01 00:00:00
467,東元,1504,Q,21.90,8.28,3.27,0.51,34.49,16.32,198.24,131.08,1.25,0.86,0.17,35.91,2022-03-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16367,川湖,2059,Y,61.28,50.12,46.93,16.86,20.14,113.88,745.17,696.78,3.34,2.31,28.38,172.47,2023-12-01 00:00:00
16384,宜鼎,5289,Q,33.21,16.62,12.30,3.74,23.70,133.88,377.70,302.03,1.59,1.25,2.97,79.61,2023-12-01 00:00:00
16385,宜鼎,5289,Y,34.23,16.60,13.76,16.04,23.70,202.65,377.70,302.03,6.03,4.72,12.98,79.61,2023-12-01 00:00:00
16472,欣興,3037,Q,17.55,5.41,11.36,3.05,55.42,25.30,195.67,163.73,1.23,1.99,1.91,59.49,2023-12-01 00:00:00


In [44]:
# 檢查數量是否正確
print(ai_stock_exchange_data['stock_symbol'].nunique())
print(ai_stock_PKTD['stock_symbol'].nunique())
print(ai_stock_chips['stock_symbol'].nunique())
print(ai_stock_fin_state['stock_symbol'].nunique())

48
48
48
48


In [45]:
for i in range(len(AI_stock_label)):
  print(AI_stock_label[i], " ", len(data['微股力個股交易數據-2年'][data['微股力個股交易數據-2年']['stock_symbol'] == AI_stock_label[i]]))

1504   485
2059   485
2308   485
2312   485
2317   485
2330   485
2353   485
2356   485
2357   485
2368   478
2376   485
2377   485
2382   485
2383   485
2395   485
2439   485
2449   485
2453   485
2454   485
3029   485
3037   485
3044   485
3078   485
3169   485
3227   485
3231   485
3324   485
3443   485
3653   485
3661   485
4966   485
5274   485
5289   485
5351   485
5439   485
6166   485
6214   485
6274   485
6414   485
6515   485
6643   485
6669   485
6689   350
6811   374
6841   241
6902   151
6933   71
8210   485


### 中文字處理，先以東元（1504）來做嘗試

In [46]:
!gdown 14H6hQylHJZLgIdtZC-AMXxukGOtWl4V7
# 這邊是使用 cool 上的 stopwords.txt 檔來用

Downloading...
From: https://drive.google.com/uc?id=14H6hQylHJZLgIdtZC-AMXxukGOtWl4V7
To: /content/stopwords_zh.txt
100% 13.3k/13.3k [00:00<00:00, 40.7MB/s]


In [47]:
# 字串以正則化處理去除中文字元以外的字元
# def clearSentence(sentence):
#     return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

with open('stopwords_zh.txt', 'r', encoding = 'utf-8') as file:
    stopwords = file.read().splitlines()
file.close()

stopwords += read_stopwords_list()
stopwords += ['記者', '報導']
stopwords += ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ']
print(len(stopwords))

3001


In [48]:
company = ['東元']

In [49]:
def where_key_words_in(df):
  result = pd.DataFrame()
  for i in range(len(company)):
    temp = df[df['title'].str.contains(company[i]) | df['content'].str.contains(company[i])]
    result = pd.concat([result, temp], ignore_index=True)
  return result

In [50]:
news_1504_part1 = where_key_words_in(data['內容數據新聞1'])
news_1504_part2 = where_key_words_in(data['內容數據新聞2'])
news_1504_part3 = where_key_words_in(data['內容數據新聞3'])

news_1504 = pd.concat([news_1504_part1, news_1504_part2, news_1504_part3], ignore_index=True)
print("news that contains:", len(news_1504))


forum_mobile01_1504_part1 = where_key_words_in(data['討論數據mobile01-1'])
forum_mobile01_1504_part2 = where_key_words_in(data['討論數據mobile01-2'])

forum_mobile01_1504 = pd.concat([forum_mobile01_1504_part1, forum_mobile01_1504_part2], ignore_index=True)
print("mobile01 forum that contains:", len(forum_mobile01_1504))


forum_Dcard_1504 = where_key_words_in(data['討論數據dcard'])
print("Dcard forum that contains:", len(forum_Dcard_1504))

forum_ptt_1504 = where_key_words_in(data['討論數據ptt'])
print("ptt forum that contains:", len(forum_ptt_1504))

news that contains: 1671
mobile01 forum that contains: 35
Dcard forum that contains: 196
ptt forum that contains: 634


In [59]:
#ngram_range n-gram 的 range, max_features 以詞頻篩選
TFIDF_vectorizer = TfidfVectorizer(stop_words=stopwords, tokenizer=jieba.lcut, max_features= 100, ngram_range = (1,2),max_df = 1., min_df = 10)
TFIDF_vectors = TFIDF_vectorizer.fit_transform(news_1504_part1['content'].values)
result_TFIDF_vectors = np.float16(TFIDF_vectors.toarray())
df = pd.DataFrame(result_TFIDF_vectors, columns=TFIDF_vectorizer.get_feature_names_out())

df.to_excel('TFIDF_output.xlsx', engine='openpyxl')
df

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['##', '±', '÷', 'β', 'δ', 'λ', 'ξ', 'ψ', 'в', '′', '″', 'ⅲ', '∈', '∧', '∪', '─', '☆', '一年', '一日', '一步', '三天', '三番', '不及', '不擇', '不知', '不覺', '主義', '之間', '二話', '今以', '介', '佈', '何樂', '例外', '保留地', '倒', '借儻', '儘', '儻', '元', '先', '兩次', '兩頭', '兼', '前', '勿', '反過', '唷', '啪', '啷', '喔', '噠', '噸', '噹', '基', '外', '多年', '夠瞧', '大張', '大面', '天', '天窗', '奈何', '妳', '始', '宣', '實實', '寬', '屆', '屬', '常言', '庶', '彈指', '得天', '從古', '從古到', '從無到', '復', '所知', '所述', '手段', '抗拒', '挨家', '挨戶', '挨門', '接下', '接連', '換句', '換言', '數', '新', '旗鼓', '早', '昉', '晚', '暗地', '更進', '期以', '未', '末', '次', '歸根', '死', '每刻', '每時', '無阻', '特', '特別', '獨厚', '甚', '皆', '

,,,10,11,111,12,1504,2021,2022,br,...,電動車,電機,電池,需求,預估,預期,預計,風電,馬,高
0,0.667969,0.000000,0.226562,0.000000,0.281738,0.000000,0.163452,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.632324,0.000000,0.000000,0.000000,0.000000,0.000000,0.110535,0.0,0.000000,0.196655,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.445801,0.121826,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.247559,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.205811,0.883301,0.000000,0.128174,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.090393,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.105957,0.000000,0.000000,0.081787,0.0,0.000000,0.000000,0.230591,0.000000,0.101440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,0.140747,0.000000,0.000000,0.055328,0.000000,0.053589,0.034454,0.0,0.036804,0.061310,...,0.000000,0.090698,0.000000,0.086426,0.0,0.000000,0.000000,0.000000,0.115845,0.053589
355,0.187988,0.000000,0.106262,0.123169,0.000000,0.000000,0.000000,0.0,0.081970,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.121826,0.000000,0.000000,0.000000
356,0.236450,0.000000,0.000000,0.154907,0.000000,0.150024,0.000000,0.0,0.103088,0.000000,...,0.783203,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.150024
357,0.194336,0.046478,0.000000,0.047760,0.000000,0.000000,0.029739,0.0,0.063538,0.000000,...,0.627930,0.156616,0.425049,0.000000,0.0,0.048035,0.000000,0.000000,0.150024,0.000000


### 新聞預測漲跌分類

In [22]:
# 新聞預測漲跌分類
exchange_data_1504 = data['微股力個股交易數據-2年'][data['微股力個股交易數據-2年']['stock_symbol'].isin(['1504'])]
exchange_data_1504.reset_index(drop=True, inplace=True)
exchange_data_1504

news_1504['post_time'] = pd.to_datetime(news_1504['post_time'])
exchange_data_1504['date'] = pd.to_datetime(exchange_data_1504['date'])
news_1504['poss_rise_or_drop'] = 0

datetime.date(news_1504.loc[0, 'post_time'])
datetime.date(exchange_data_1504.loc[0, 'date']) + timedelta(days=1)

<ipython-input-22-b90f285560c0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_data_1504['date'] = pd.to_datetime(exchange_data_1504['date'])


datetime.date(2022, 3, 2)

In [23]:
for i in range(len(exchange_data_1504)):
  for j in range(len(news_1504)):
    if (datetime.date(exchange_data_1504.loc[i, 'date']) == datetime.date(news_1504.loc[j, 'post_time'])+ timedelta(days=1)):
      if (exchange_data_1504.loc[i, 'open'] > exchange_data_1504.loc[i, 'close']):
        news_1504.loc[j, 'poss_rise_or_drop'] = -1
      elif (exchange_data_1504.loc[i, 'open'] < exchange_data_1504.loc[i, 'close']):
        news_1504.loc[j, 'poss_rise_or_drop'] = 1